In [1]:
import json
import ROOT
from analysis_framework import Dataset
from WWAnalysis import WWAnalysis
import os

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x7c25aa0


In [2]:
ROOT.EnableImplicitMT(12)

In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad

In [4]:
prod = True
no_rvec = False
# output_collections = r"(\w*sqme\w*)|(weight\w*)"
dataset_path = "stage2-test.json"
# # output_path = "data/snapshots/stage2-test"
# output_meta = "stage2-test.json"
if prod:
    dataset_path = "stage2-full.json"
    # output_path = "data/snapshots/stage2"
    # output_meta = "stage2-full.json"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = WWAnalysis(dataset)
analysis.init_categories()

In [7]:
alt_configs = {
    "alt1": {
        "g1z": 1.001,
        "ka": 1.000,
        "la": 0.000,
    },
    "alt2": {
        "g1z": 1.000,
        "ka": 1.001,
        "la": 0.000,
    },
    "alt3": {
        "g1z": 1.000,
        "ka": 1.000,
        "la": 0.001,
    },
}
# analysis.initialise_omega_wrappers(alt_configs)

In [8]:
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]
analysis.set_signal_categories(signal_category)

True


In [9]:
print(json.dumps(analysis._categories, indent=2))

{
  "4f_sw_sl_signal": [
    "4f_sw_sl_eLpL_signal",
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpR_signal",
    "4f_sw_sl_eRpL_signal"
  ],
  "4f_sl_bkg": [
    "4f_zz_sl_eLpR",
    "4f_zz_sl_eRpL",
    "4f_zznu_sl_eLpR",
    "4f_zznu_sl_eRpL",
    "4f_sze_sl_eLpL",
    "4f_sze_sl_eLpR",
    "4f_sze_sl_eRpR",
    "4f_sze_sl_eRpL",
    "4f_sznu_sl_eLpR",
    "4f_sznu_sl_eRpL",
    "4f_ww_sl_eLpR",
    "4f_ww_sl_eRpL",
    "4f_sw_sl_eLpL_bkg",
    "4f_sw_sl_eLpR_bkg",
    "4f_sw_sl_eRpR_bkg",
    "4f_sw_sl_eRpL_bkg"
  ],
  "4f_not_sl": [
    "4f_zz_h_eLpR",
    "4f_zz_h_eRpL",
    "4f_zz_l_eLpR",
    "4f_zz_l_eRpL",
    "4f_zzorww_h_eLpR",
    "4f_zzorww_l_eLpR",
    "4f_zzorww_l_eRpL",
    "4f_zznu_l_eLpR",
    "4f_zznu_l_eRpL",
    "4f_sznu_l_eLpR",
    "4f_sznu_l_eRpL",
    "4f_sze_l_eLpL",
    "4f_sze_l_eLpR",
    "4f_sze_l_eRpR",
    "4f_sze_l_eRpL",
    "4f_szeorsw_l_eLpL",
    "4f_szeorsw_l_eLpR",
    "4f_szeorsw_l_eRpR",
    "4f_szeorsw_l_eRpL",
    "4f_sw_l_eLpL",
    "4f_sw_l_

In [10]:
analysis.Define("O_1", "1e3 * (reco_sqme_12_nominal + reco_sqme_21_nominal - reco_sqme_12_alt1 - reco_sqme_21_alt1) / (reco_sqme_12_nominal + reco_sqme_21_nominal)")
analysis.Define("O_2", "1e3 * (reco_sqme_12_nominal + reco_sqme_21_nominal - reco_sqme_12_alt2 - reco_sqme_21_alt2) / (reco_sqme_12_nominal + reco_sqme_21_nominal)")
analysis.Define("O_3", "1e3 * (reco_sqme_12_nominal + reco_sqme_21_nominal - reco_sqme_12_alt3 - reco_sqme_21_alt3) / (reco_sqme_12_nominal + reco_sqme_21_nominal)")
analysis.Define("weight_nominal", "1.0")

In [11]:
analysis.book_histogram_1D("weight_alt1", "weight_alt1", ("", ";weight_alt1", 50, 0.8, 1.2), categories=signal_category)

In [ ]:
OO_configs = {
    "O_1": ("", ";O_1", 50, -2., 4.),
    "O_2": ("", ";O_2", 50, -3., 3.),
    "O_3": ("", ";O_3", 50, -5., 5.),
}
for name, conf in OO_configs.items():
    analysis.book_histogram_1D(name, name, conf)

In [13]:
weights = [f"weight_{name}" for name in ["nominal"] + list(alt_configs.keys())]
analysis.book_templates(OO_configs, weights, signal_category)

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x16880240


In [14]:
%%time
analysis.run()

CPU times: user 1.99 s, sys: 169 ms, total: 2.16 s
Wall time: 2.03 s


In [15]:
%jsroot on
analysis.draw_histogram("weight_alt1", categories=signal_category)
analysis.draw_histogram("O_1")
analysis.draw_histogram("tmpl_O_1", categories=signal_category, draw_opt="e1")
analysis.draw_histogram("O_2")
analysis.draw_histogram("tmpl_O_2", categories=signal_category, draw_opt="e1")
analysis.draw_histogram("O_3")
analysis.draw_histogram("tmpl_O_3", categories=signal_category, draw_opt="e1")

(<cppyy.gbl.THStack object at 0x19fe1c80>,
 <cppyy.gbl.TCanvas object at 0xa5d0b00>)

In [16]:
confs = ["nominal", "alt1", "alt2", "alt3"]
varied_histos_per_df = analysis._varied_histograms["tmpl_O_1"]
varied_histos = varied_histos_per_df["4f_sw_sl_eLpR_signal"]
print(varied_histos.GetKeys())
ratios = {}
for c in confs[1:]:
    ratios[c] = varied_histos[f"weight:{c}"] / varied_histos["nominal"]

{ "nominal", "weight:alt1", "weight:alt2", "weight:alt3" }


In [17]:
%jsroot on
c_ratio1 = ROOT.TCanvas()
ratios["alt1"].Draw("hist")
c_ratio1.Draw()